In [18]:
from src import Tensor,cross_entropy,Adam,SGD,Linear, Module, Sequential, ReLU,dataloader
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [19]:
import sklearn.datasets
train_X, train_y = sklearn.datasets.load_digits(return_X_y=True)

In [20]:
class Net(Module):
    def __init__(self, outc):
        super().__init__()
        self.seq = Sequential(
            Linear(None, 64, bias=True),
            ReLU(),
            Linear(None, 64*2, bias=True),
            ReLU(),
            Linear(None, outc),
        )

    def forward(self, x):
        x = self.seq(x)
        return x


net = Net(len(np.unique(train_y)))
optimizer = Adam(net.trainable_params(), lr=0.001,amsgrad=True)
for epoch in range(10):  # loop over the dataset multiple times
    losses = []
    accs = []
    for X, y in dataloader(64*2, train_X, train_y):

        # forward + backward + optimize
        outputs = net.forward(X)
        loss = cross_entropy(outputs, y)

        optimizer.zero_grad()
        loss.backward()
        assert loss.item() > 0, loss
        optimizer.step()

        losses.append(loss.item())
        accs.append((outputs.data.argmax(-1) == y.data).astype(float).mean())
    print(
        f'[Epoch {epoch + 1:3}] loss: {np.mean(losses):.3f} accuracy: {np.mean(accs)*100:3.2f}')

print('Finished Training')

[Epoch   1] loss: 3.366 accuracy: 52.55
[Epoch   2] loss: 0.468 accuracy: 86.27
[Epoch   3] loss: 0.237 accuracy: 93.04
[Epoch   4] loss: 0.121 accuracy: 95.94
[Epoch   5] loss: 0.075 accuracy: 97.55
[Epoch   6] loss: 0.049 accuracy: 98.59
[Epoch   7] loss: 0.037 accuracy: 99.17
[Epoch   8] loss: 0.030 accuracy: 99.38
[Epoch   9] loss: 0.024 accuracy: 99.58
[Epoch  10] loss: 0.021 accuracy: 99.79
Finished Training


In [21]:
(train_y == net(Tensor(train_X)).data.argmax(-1)).mean()

0.998330550918197